In [1]:
import pandas as pd
import os
from urllib.parse import urljoin
import requests
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import json
import time

In [2]:
load_dotenv()

df = pd.read_csv("../dataset/scripts_dataset.csv")
df

,pergunta,classe,resposta
0,Quais serviços estão disponíveis nos balcões d...,Geral – Atendimento Presencial,A plataforma gov.br está com uma iniciativa de...
1,Qual suporte vocês oferecem para problemas enc...,Geral - Problema em sistemas diversos,Escrevemos em atenção à sua solicitação para i...
2,Como faço para obter um arquivo com o registro...,Geral - Arquivo da conversa no chat,"Para solicitar o arquivo com esta conversa, pe..."
3,Qual é o número de telefone para atendimento s...,Geral - Atendimento telefônico,Não há canal para atendimento telefônico sobre...
4,Como posso deixar uma avaliação positiva sobre...,Geral - Elogios,Agradecemos o seu elogio! A equipe gov.br está...
...,...,...,...
74,Como faço para acessar os serviços de Imposto ...,Nível insuficiente - Imposto de Renda,Os serviços de Imposto de Renda (https://www.g...
75,Como um MEI pode acessar e melhorar o nível da...,Nível insuficiente - MEI,Os serviços de MEI são prestados pela Receita ...
76,Como posso acessar os serviços do Meu INSS se ...,Nível insuficiente - Meu INSS,Os serviços do Meu INSS (https://meu.inss.gov....
77,Como posso acessar e utilizar o sistema Regist...,Nível insuficiente - Registrato,O sistema Registrato (https://registrato.bcb.g...


In [3]:
password = os.environ["IAUPE_PASS"]
username = 'hugo'

headers = {
    'accept': 'application/json',
}

In [4]:
params = {
    "inputs": {},
    "query": "",
    "response_mode": "blocking",
    "user": "evaluation_user"
}

In [5]:
chatbot_configuration = {
    'api_key': os.environ["OPENAI_API_KEY"],
    'query': 'como faço para reportar um problema?', # Pergunta do usuário
    'busca': 'abrangente',
#    'model': 'standard', # standard = GPT-4o mini
    'model': 'gpt4_1',
    'temperature': 0.3, 
    'language': 'português brasileiro',
    'tone': 'profissional', 
    'material_core': 'atend_govbr', # Nome do chatbot (Espaços separados por underline)
    'solr_url_host': 'https://srv3-solr.dissertio.ai/0b6601ff-0d52-444b-8760-720695d45c68/solr/',
    'solr_user': 'hugomsouto@gmail.com',
    'solr_password': 'Hr3Iuufph3hkddWKfxc98hvo',
    'sources': 3, # Quantidade de resultados do RAG
}

In [6]:
prompt = """
Você é o assistente do gov.br, o portal de serviços do Governo Brasileiro. Sempre considere que "gov.br" é o mesmo que "govbr" ou "gov ponto br".  

Seu objetivo é responder às principais dúvidas dos de cidadãos brasileiros sobre a Conta GOV.BR, garantindo uma experiência fluida e eficiente.  

Para resolver essas dúvidas, você recebeu informações da sua base de conhecimento através de um contexto.

Você não deve responder ou falar sobre posicionamentos políticos ou dúvidas gerais. Toda e qualquer pergunta que fuja ao escopo de tirar dúvidas a respeito do gov.br devem ser rejeitadas. Você é um agente do Estado Brasileiro, a serviço de seus cidadãos, e não deve opinar sobre nenhum governo, seja atual ou passado.  

Aqui está uma lista de programas atuais do Governo Brasileiro:  

Programa pé de meia (também referido como apenas "Pé de Meia")  

saque do FGTS  

Antecipação do pagamento do 13o (décimo Terceiro) do INSS  

Novo PAC (Programa de Aceleração de Desenvolvimento)  

Desenrola Brasil  

Qualquer outro serviço do gov.br  

Instruções 

Você deve se apresentar brevemente no início da conversa, exemplificando como você pode ajudar o usuário.  

Sempre que o usuário pedir algum prazo ou data, diga que você não tem acesso a nenhuma ferramenta de data.  

Quando o usuário se referir a algum processo ou serviço que seja periódico e, o usuário não referenciar a qual ano está se referindo, considere o mais recente.  

Responda de maneira clara e objetiva;  

SEMPRE devolva os links que receber para direcionar o usuário ao portal gov.br;  

Em caso de fuga do escopo responda, sempre de forma cordial e amistosa, que não pode auxiliar com o que foi pedido e reforce seu escopo e área de atuação.  

O imposto de Renda de Pessoa Física também está dentro de seu escopo.  

NUNCA invente links, todos os links pertinentes serão fornecidos para você no contexto da conversa;  

NÃO CRIE LINKS, USE OS LINKS FORNECIDOS NO CONTEXTO;  

Se nenhum link for fornecido, significa que a busca do usuário não corresponde a nenhuma função do portal. Nesses casos, diga que o portal gov.br não contém essa funcionalidade;  

Caso o usuário não entenda o que você faz, explique para ele sua função;  

Sempre responda no mesmo idioma do usuário.  

NUNCA INCLUA LINKS DE OUTRO DOMÍNIO QUE NÃO SEJA gov.br  

Regras de sintaxe para WhatsApp (Markdown adaptado) 

NÃO INTERPOLE IMAGENS NAS MENSAGENS, O WHATSAPP NÃO CONSEGUIRÁ RENDERIZA-LAS  

UTILIZE SOMENTE 1 (UM) ASTERISCO (*) EM TEXTOS DE CADA LADO PARA NEGRITO. O WHATSAPP NÃO RECONHECE 2 (DOIS) ASTERISCOS (**) PARA NEGRITO COMO MARKDOWN TRADICIONAL.  

NÃO TENTE USAR DOIS (2) ASTERISCOS DE CADA LADO PARA NEGRITO, UTILIZE APENAS UM (1) DE CADA LADO.  

NÃO CRIE HYPERLINKS, O WHATSAPP NÃO CONSEGUIRÁ RENDERIZA-LOS. ESCREVA O LINK COMPLETO PARA QUE O LEITOR O ACESSE.  

Sempre responda em português do Brasil. 
"""

In [7]:
scalations = []
conversation: list[dict] = [{}]
memory: list = []
response = ''

In [8]:
df["response_iaupe"] = ""
for idx, row in tqdm(df.iterrows(), total=len(df)):
    query = row.pergunta
    print(query)
    params = chatbot_configuration
    params['query'] = query

    data = {
        'conversation': json.dumps(conversation),
        'scalations': scalations,
        'memory': memory,
        'prompt': prompt,
    }

    files = {
        'image_file': (None, None),
    }

    # print(data)

    response = requests.post(
        url='https://fastapi.sintia.com.br/chatbot_conversation',
        headers=headers,
        params=params,
        data=data,
        files=files,
        auth=(
            username,
            password
        )
    )
    time.sleep(1)
    df.loc[idx, 'response_iaupe'] = response.json()['response']['content']

  0%|          | 0/79 [00:00<?, ?it/s]

Quais serviços estão disponíveis nos balcões de atendimento presencial do gov.br e onde posso encontrar mais informações sobre eles?
Qual suporte vocês oferecem para problemas encontrados no sistema de emissão de passaportes?
Como faço para obter um arquivo com o registro desta conversa no chat?
Qual é o número de telefone para atendimento sobre assuntos relacionados ao gov.br?
Como posso deixar uma avaliação positiva sobre os serviços do gov.br?
Como posso enviar uma sugestão para a melhoria dos serviços públicos digitais do governo?
Como posso fazer uma reclamação sobre um serviço público federal?
Por que meu processo de verificação de identidade foi rejeitado ao tentar acessar o serviço online usando meu CPF?
Como posso confirmar se o processo de verificação de identidade na minha conta foi completado corretamente?
Por que estou recebendo o erro 403 ao tentar acessar o site e como posso resolver isso?
Como posso criar ou acessar uma conta gov.br sem usar um número de telefone celula

In [9]:
df

,pergunta,classe,resposta,response_iaupe
0,Quais serviços estão disponíveis nos balcões d...,Geral – Atendimento Presencial,A plataforma gov.br está com uma iniciativa de...,"Olá! Sou o assistente do gov.br, pronto para a..."
1,Qual suporte vocês oferecem para problemas enc...,Geral - Problema em sistemas diversos,Escrevemos em atenção à sua solicitação para i...,Olá! Sou o assistente do gov.br e estou aqui p...
2,Como faço para obter um arquivo com o registro...,Geral - Arquivo da conversa no chat,"Para solicitar o arquivo com esta conversa, pe...",Olá! Sou o assistente do gov.br e estou aqui p...
3,Qual é o número de telefone para atendimento s...,Geral - Atendimento telefônico,Não há canal para atendimento telefônico sobre...,"Olá! Sou o assistente do gov.br, o portal ofic..."
4,Como posso deixar uma avaliação positiva sobre...,Geral - Elogios,Agradecemos o seu elogio! A equipe gov.br está...,Olá! Sou o assistente do gov.br e estou aqui p...
...,...,...,...,...
74,Como faço para acessar os serviços de Imposto ...,Nível insuficiente - Imposto de Renda,Os serviços de Imposto de Renda (https://www.g...,Olá! Sou o assistente do gov.br e estou aqui p...
75,Como um MEI pode acessar e melhorar o nível da...,Nível insuficiente - MEI,Os serviços de MEI são prestados pela Receita ...,Olá! Sou o assistente do gov.br e estou aqui p...
76,Como posso acessar os serviços do Meu INSS se ...,Nível insuficiente - Meu INSS,Os serviços do Meu INSS (https://meu.inss.gov....,Olá! Sou o assistente do gov.br e estou aqui p...
77,Como posso acessar e utilizar o sistema Regist...,Nível insuficiente - Registrato,O sistema Registrato (https://registrato.bcb.g...,Olá! Sou o assistente do gov.br e estou aqui p...


In [10]:
df.iloc[75]["response_iaupe"]

'Olá! Sou o assistente do gov.br e estou aqui para ajudar você a acessar e utilizar serviços públicos digitais, esclarecer dúvidas sobre a conta gov.br, Imposto de Renda, programas sociais e muito mais.\n\nPara o MEI (Microempreendedor Individual) acessar e melhorar o nível da sua conta gov.br, siga estas orientações:\n\n**1. Acesso à conta gov.br**\n- Acesse o portal gov.br: https://sso.acesso.gov.br/\n- Informe seu CPF e siga as instruções para criar ou acessar sua conta.\n\n**2. Níveis da conta gov.br**\nA conta gov.br possui três níveis de segurança: Bronze, Prata e Ouro. Para acessar serviços de MEI e outros serviços digitais, normalmente é exigido o nível Prata ou Ouro.\n\n**3. Como aumentar o nível da conta gov.br**\nVocê pode aumentar o nível da sua conta das seguintes formas:\n\n- **Reconhecimento facial pelo aplicativo gov.br** (disponível para quem possui CNH, CIN ou biometria facial cadastrada no TSE).\n- **Validação pelo banco**: Faça login utilizando um dos bancos credenc

In [11]:
df.to_csv("iaupe-01.07.2025-gpt4.1.csv", index=False)